In [18]:
// move assembly jar from .target to root first
import $cp.`zy-test-protobuf-assembly-1.0-SNAPSHOT.jar`

import com.tubitv.rpc.analytics._

import $cp.$                                           


import com.tubitv.rpc.analytics._

In [10]:
val builder = TestEvent.newBuilder()

val mthd =builder.getClass().getDeclaredMethods().filter(_.getName().matches("^setStr([2-9]|\\d{2,})$")).filter(m => m.getParameterTypes()(0).getSimpleName.equals("TestField"))

val union_builder = TestUnion.newBuilder().setValue(88)
val field_builder = TestField.newBuilder().setValue("test").setEnumValueValue(0)
mthd.foreach(m => { m.invoke(builder, field_builder.setUnionValue(union_builder.build()).build()) })

builder.addStr1(field_builder.setUnionValue(union_builder.build()).build())

val event = builder.build()

builder: TestEvent.Builder = str1 {
  value: "test"
  unionValue {
    value: 88
  }
}
str2 {
  value: "test"
  unionValue {
    value: 88
  }
}
str3 {
  value: "test"
  unionValue {
    value: 88
  }
}
str4 {
  value: "test"
  unionValue {
    value: 88
  }
}
str5 {
  value: "test"
  unionValue {
    value: 88
  }
}
str6 {
  value: "test"
  unionValue {
    value: 88
  }
}
str7 {
  value: "test"
  unionValue {
...
mthd: Array[java.lang.reflect.Method] = Array(
  public com.tubitv.rpc.analytics.TestEvent$Builder com.tubitv.rpc.analytics.TestEvent$Builder.setStr2(com.tubitv.rpc.analytics.TestField),
  public com.tubitv.rpc.analytics.TestEvent$Builder com.tubitv.rpc.analytics.TestEvent$Builder.setStr3(com.tubitv.rpc.analytics.TestField),
  public com.tubitv.rpc.analytics.TestEvent$Builder com.tubitv.rpc.analytics.TestEvent$Builder.setStr4(com.tubitv.rpc.analytics.TestField),
  public com.tubitv.rpc.analytics.TestEvent$Builder com.tubitv.rpc.analytics.TestEvent$Builder.setStr5(com.tubitv.

In [11]:
import java.io.File
import java.io.FileOutputStream

val file = new File("output.pb")

file.createNewFile()

val fos = new FileOutputStream(file)

event.writeTo(fos)



import java.io.File

import java.io.FileOutputStream


file: File = output.pb
res10_3: Boolean = false
fos: FileOutputStream = java.io.FileOutputStream@2d8427be

In [12]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`
import $ivy.`org.apache.spark:spark-protobuf_2.12:3.4.0`

import $ivy.$                                  

import $ivy.$                                           

In [13]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@7ea7bb7a

In [14]:
import org.apache.spark.sql.functions._

val df = spark.read.format("binaryFile").load("output.pb")
val df2 = df.select(col("content"))

df2.printSchema()
df2.show()

root
 |-- content: binary (nullable = true)



show at cmd13.sc:7 
 (kill)

0 / 1

+--------------------+
|             content|
+--------------------+
|[0A 0A 0A 04 74 6...|
+--------------------+



import org.apache.spark.sql.functions._


df: DataFrame = [path: string, modificationTime: timestamp ... 2 more fields]
df2: DataFrame = [content: binary]

In [15]:
import spark.implicits._
case class RawData(content: Array[Byte])

val rawDs = df2.as[RawData]

import spark.implicits._

defined class RawData
rawDs: Dataset[RawData] = [content: binary]

In [16]:
// implicit val encoder = Encoders.kryo[TestEvent]
// val ds = rawDs.map(r => TestEvent.parseFrom(r.content))
// ds.map(_.getStr2).show()

import org.apache.spark.sql.protobuf.functions._
val df = rawDs.select(from_protobuf($"content", "com.tubitv.rpc.analytics.TestEvent").as("event"))

import org.apache.spark.sql.protobuf.functions._

df: DataFrame = [event: struct<str1: array<struct<value:string,enumValue:string,unionValue:struct<value:int>>>, str2: struct<value: string, enumValue: string ... 1 more field> ... 298 more fields>]

In [17]:
df.schema

res16: types.StructType = StructType(
  StructField(
    "event",
    StructType(
      StructField(
        "str1",
        ArrayType(
          StructType(
            StructField("value", StringType, true, {}),
            StructField("enumValue", StringType, true, {}),
            StructField(
              "unionValue",
              StructType(StructField("value", IntegerType, true, {})),
              true,
              {}
            )
          ),
          false
        ),
        true,
        {}
      ),
      StructField(
        "str2",
        StructType(
          StructField("value", StringType, true, {}),
          StructField("enumValue", StringType, true, {}),
          StructField(
            "unionValue",
            StructType(StructField("value", IntegerType, true, {})),
            true,
            {}
          )
        ),
        true,
        {}
      ),
      StructField(
        "str3",
...